
_Inteligencia de Cliente (CRM)_

_Máster Universitario en Inteligencia de Negocio_

# Toma de decisiones estratégicas I: Análisis descriptivo de los datos(EDA) Adidas Sales


 En este caso práctico se lleva a cabo un análisis exploratorio e los datos (EDA) de las ventas de Adidas. El objetivo que persigue este análisis es identificar información clave que ayude a lanzar nuevos productos, mejorar el rendimiento de las ventas y a optimizar sus estrategias comerciales. Este análisis nos permitirá responder preguntas como:

- ¿Qué productos están destacando en ventas y cuáles tienen un rendimiento inferior?
- ¿Qué tiendas están registrando buenas ventas y cuáles se están quedando atrás?
- ¿Tiene el margen de beneficio un impacto significativo en las ventas?
- ¿Qué canal es más efectivo para las ventas?
- ¿Qué productos son más rentables en cada ubicación?
- ¿Existe un rango de precios específico que consiga mejores ventas que otros?
- ¿Qué ubicaciones de tiendas obtienen mejores y peores resultados?
- ¿Existe algún tipo de componente estacional en las ventas?
- ¿Es posible realizar una predicción de las ventas mensuales?


Para llevarlo a cabo disponemos de un conjunto de datos sobre su actividad comercial en EEUU. La descripción del conjunto de datos disponible es la siguiente:


- **Retailer**: Distribuidor. Empresas como Foot Locker, Walmart, Sports Direct, and West Gear.
- **Retailer ID**: Identificador único de cada distribuidor.
- **Invoice Date**: Fecha en la que la factura fue generada.
- **Region**: Áreas geográficas de EEUU como West, Northeast, Southeast, South, and Mid-west.
- **Product**: Categorías de productos como Men’s and Women’s Street, Athletic Footwear, Apparel.
- **Price per Unit**: Coste unitario de un artículo de un determinado producto específico.
- **Units Sold**: La cantidad de un producto vendido en un plazo determinado
- **Total Sales**: Total de ingresos por ventas durante un periodo determinado.
- **Operating Profit**: Una medida de la rentabilidad. Beneficio obtenido por el minorista de sus operaciones comerciales.
- **Operating Margin**: Una medida de la rentabilidad. Margen de beneficio obtenido por el minorista en sus operaciones comerciales.
- **Sales Method**: Los canales a través de los cuales se realizaron las ventas, incluidos los canales en tienda física (store), outlet y tienda online.


## 0. Importación de librerias y carga de datos

In [ ]:
pip install us # La librería us no viene instalada en Colab por lo que la instalamos. Se utiliza para formatear unidades, validar cadenas de texto...

In [ ]:
# Importamos las librerías necesarias
import numpy as np # Nos permite hacer calculos estadísticos como la media, mediana, hacer filtrados de datos...
import pandas as pd #Nos permite leer los datos en diferentes formatos, tratar valores faltantes, transformar datos....
from matplotlib import pyplot as plt # Permite hacer gráficos y visualizaciones como histogramas, diagramas de dispersión...
import seaborn as sns # Permite crear visualizaciones de una forma más sencilla, tiene temas prededifinidos para gráficos...
import plotly.express as px # Permite crear gráficos interactivos y hacerlo con menos código
import plotly.graph_objs as go # Permite crear gráficos interactivos y personalizar elementos del gráfico.
from statsmodels.tsa.seasonal import seasonal_decompose # Permite descomponer en series temporales.
from statsmodels.tsa.arima.model import ARIMA #Permite utilizar el modelo ARIMA para el análisis y pronóstico de series temporales.

In [ ]:
# Cargamos el dataset en Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Leemos el dataset
import io
data = pd.read_excel(io.BytesIO(uploaded['Adidas US Sales Datasets.xlsx']))
data.head(10)

## 1. Análisis de la estructura de los datos y sus principales estadísticas

En este paso vemos el número de variables que tiene el dataset, el número de observaciones o instancias, si hay celdas vacías o con valores erróneos y todo lo relacionado con la calidad de los datos. Realizar este análsis nos permitirá preparar los datos para la aplicación de los distintos algoritmos de machine lerning y alcanzar conclusiones.

In [ ]:
# Vemos el número de instancias y variables que tiene el dataset
data.shape

**Pregunta:** ¿Cuántas variables tiene el dataset? ¿Cuántas instancias tiene el dataset?

**Respuesta:** Tiene 13 variables y 9648 instancias.

In [ ]:
# Comprobamos la existencia de filas duplicadas
data.duplicated().sum()

**Pregunta:** ¿Existen filas duplicadas en el dataset?

**Respuesta:** No

In [ ]:
#Vemos qué tipo de variables contiene el dataset y qué valores no contienen nulos
data.info()

**Pregunta:** ¿Cuántas variables numéricas tiene el dataset? ¿Cuántas de tipo caracter o string?

**Respuesta:** Existen tres tipos de variables:

- Numéricas (enteras int64 ó decimales float64).
- De texto o "strings" (object).
- De tipo fecha (datetime)

In [ ]:
# Generamos estadísticas descriptivas  de los datos numéricos para tener una visión de alto nivel de la estructura  de los datos
data.describe()

**Pregunta:** ¿Cómo están distribuidos los datos? Describe en términos generales sus principales medidas , media, desviación estándar, min y max y quartiles.

**Respuesta:**
- Vemos que todas las variables tienen el mismo número de instancias, 9468 que coincide con el numero de instancias del dataset lo que nos da a entender que no hay valores faltantes o celdas vacias.
- Vemos que la media es razonable respecto a los valores minimos y maximos que adopta la variable.
- Respecto a la variable precio si observamos la distribución por cuartiles vemos que el 25% de las unidades se vendieron por debajo de 35 dólares, el 50% por debajo de 45 dólares y el 75% por debajo de 55 dólares. Podemos pensar que la demanda se ha concentrado entre los 35 y 55 dólares por ejemplo.
- En relación al total de ventas si observamos también los cuartiles que el 25 % de las ventas totales (ventas totales durante un periodo determinado) han sido iguales o inferiores a 4.254 dólares, el 50% se ha situado por debajo de 9576 dólares y el 75% por debajo de 150.000. A priori parece que la diferencia entre los rangos intercuartílicos 25 y 75 existe una diferencia considerable lo que puede indicar una distribución no homogénea en las ventas.

In [ ]:
#Dibujamos los histogramas de las variables numéricas para conocer su distribución
data.hist(figsize=(20, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
#Analizamos las estadísticas de la variable de interés  "Price per Unit" . Esta variable la consideramos de interés porque será útil para para predecir el precio en un futuro
data['Price per Unit'].describe()

**Pregunta:** ¿Cómo es la distribución de la variable Price per Unit? Describe en términos generales sus principales medidas , media, desviación estándar, min y max y quartiles.

**Respuesta:** Si nos fijamos por ejemplo en la variable que representa el precio por unidad, vemos que la media de precios de los artículos es de 45,2 dólares , con un máximo de 110 dólares y un mínimo de 7 dólares.

In [ ]:
#Analizamos la variable "Product". En esta ocasión necesitamos sabersaber si es categórica o no. Esto es importante si deseamos por ejemplo si queremos predecir que tipo de producto se vende más en una ubicación
print('Product: ',data['Product'].unique())

**Pregunta:** ¿Piensas que la variable Product es una variable categórica?

**Respuesta:** Si, es una variable de tipo string (object) que solo puede tomar un limitado conjunto de valores. Esta comprobación la hacemos dado que si pretendemos utilizarla para realizar futuras predicciones de clasificación.

In [ ]:
# Realizamos las tareas de limpieza del dataset. Para elllo visualizamos el total de valores nulos de cada variable del dataset. Si los hay tendremos que valorar si los eliminamos o sustituimos.
data.isnull().sum()

**Pregunta:** ¿Existen valores nulos en el dataset?

**Respuesta:** No

**Pregunta**¿Qué deberíamos hacer si existieran?

**Respuesta**: Tendríamos que ver el motivo por el que esos valores aparecen como nulos y valorar si los eliminamos o no. Si una variable tuviera muchos nulos podríamos valorar eliminarlos, por ejemplo a partir de un 10%. Si no podriamos sustituirlos por su media o por su mediana. Dependiendo del tipo de variable,la distribución de los datos o el tipo de análisis puede ser más o menos util utilizar la una o la otra. Por ejemplo si hablamos de precio podriamos sustituir por la media, en el caso de tipo de producto lo haríamos por la mediana.

In [ ]:
# Seleccionar solo las columnas numéricas
df_numeric = data.select_dtypes(include=['float64', 'int64'])

#Realizamos la matriz de correlación entre las variables
correlation_matrix = df_numeric.corr(method ='pearson')

# Crear un mapa de calor con seaborn
plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlación')
plt.show()

## 2. Análisis del rendimiento de las ventas: ¿Qué productos están destacando en ventas y cuáles no?

In [ ]:
# Primero vamos a averiguar los ingresos totales
total_revenue = data['Total Sales'].sum()
total_revenue

In [ ]:
# Agrupamos los productos en función de sus ingresos por las ventas totales (Total Sales) y de las unidades vendidas de cada producto (Units Sold)
product_sales = data.groupby('Product').agg({'Total Sales': 'sum', 'Units Sold': 'sum'}).reset_index()

# Ordenamos el resultado de los productos en función de los ingresos por las ventas totales de cada uno en orden descendente para identificar los productos con mejores resultados.
top_performing_products = product_sales.sort_values(by='Total Sales', ascending=False).head()

# Visualizamos los resultados
print("Top-Performing Products:")
top_performing_products

In [ ]:
# Ahora, a partir del resultado anterior, mostramos los últimos productos de la lista que muestran los productos con menos ingresos por ventas y por tanto los productos con peores resultados.
underperforming_products = product_sales.sort_values(by='Total Sales', ascending=False).tail()

print("\nUnderperforming Products:")
underperforming_products

**Pregunta:** ¿Por cuál de los productos obtenemos más ingresos? ¿Y menos?

**Respuesta:** Las zapatillas de vestir para hombre "Men's Street Footware" es el productopor el que se han obtenido un mayor número de ingresos en ventas con unas ventas de 208.826.244 dólares ibtenidos con la venta de 59.3320 unidades.

El producto con peores resultados es calzado deportivo para mujer "Women's Athletic Footwear", con un total de ingresos por ventas de 106.631.896 dólares a partir de la venta de 317236 unidades.

Cabe destacar que nos estamos fijando en los ingresos totales por las ventas de cada producto. Si nos fijamos en el número de unidades vendidas vemos que por ejemplo que hay más unidades vendidas del producto "Men's Athletic Footwear" (435526) que del producto "Women's Apparel" (433827), pero sus ingresos totales son menores, normalmente por un precio por unidad menor. Algo que podemos comprobar a partir de la siguiente celda.

In [ ]:
# Agrupamos los productos en función de sus ingresos por las ventas totales (Total Sales) y de las unidades vendidas de cada producto (Units Sold)
data.groupby('Product').agg({'Price per Unit': 'mean'}).reset_index()

## 3. Análisis del mercado regional: ¿Qué tiendas registran mayores ventas y cuáles menos?

In [ ]:
# Visualizamos de nuevo las primeras filas del dataset
data.head()

Para este análisis, vamos a descubrir cuáles son las tienda con mejores y peores resultados en cada una de las ciudades mencionadas. Para ello en primner lugar vamos a ver cuántas ciudades existen y, a continuación, cuántas tiendas existen en cada una de esas ciudades.

In [ ]:
#Elegimos la variable City para ver los distintos valores que adopta. Con esto vemos las ciudades distintas que hay en el dataset
print('City: ',data['City'].unique())
#Mostramos también el número de ciudades distintas
print('Number of unique cities: ',data['City'].nunique())

In [ ]:
# Agrupamos cada ciudad ('City') con el número de minoristas existentes en cada ciudad ('Retailer'). Y sumamos el número de unidades vendidas ('Units Sold') para cada una de ellas
three_columns_grouped = data.groupby(['City', 'Retailer'])['Units Sold'].sum().reset_index()

# Ordenamos los resultados en función de ombre de la ciudad y del número de unidades venididas por cada tienda (orden descendente, de mayor a menor número de unidades vendidas)
three_columns_sorted = three_columns_grouped.sort_values(by=['City', 'Units Sold'], ascending=[True, False])

# Del resultado, obtenemos los vendedores con un mayor número de unidades vendidas en cada una de las 52 ciudades
top_performers = three_columns_sorted.groupby('City').head(1)

# Del resultado, obtenemos los vendedores con un menor número de unidades vendidas en cada una de las 52 ciudades
worst_performers = three_columns_sorted.groupby('City').tail(1)

In [ ]:
# Visualizamos los resultados: vendedores con un mayor número de unidades vendidas en cada una de las 52 ciudades
print("Top Performing Retailers in Each City:")
top_performers.head(52)


In [ ]:
# Visualizamos los resultados: vendedores con un menor número de unidades vendidas en cada una de las 52 ciudades
print("\nWorst Performing Retailers in Each City:")
worst_performers.head(52)

**Respuesta:** ¿Qué conclusiones podemos sacar de estos resultados?

**Respuesta:** En el resultado obtenemos las tiendas con mejor y peor desempeño de cada ciudad en términos de unidades vendidas. Dicho de otra manera, las tiendas que mueven mayor y menor volumen en cada ciudad.

Así, por ejemplo, vemos que en New York	la tienda que más unidades vende es Foot Locker	con 72196 unidades, mientras que la que menos vende es Sports Direct con 5032 unidades.

Este último un volumen que no es muy diferente del de la tienda que más vende en Baltimore, que es Foot Locker con 9322 unidades.

## 4. Análisis del margen de beneficios: ¿Influye significativamente el beneficio operativo en las ventas?

Una forma rápida y sencilla de valorar si es posible que exista relacién entre el beneficio operativo "Operating Profit" de las tiendas y su volumen de ventas "Total Sales" o el número de unidades vendidas "Units Sold" es ver si existe correlación entre ambas variables.

In [ ]:
# Seleccionar solo las columnas numéricas
df_numeric = data.select_dtypes(include=['float64', 'int64'])

#Realizamos la matriz de correlación entre las variables
correlation_matrix = df_numeric.corr(method ='pearson')

# Crear un mapa de calor con seaborn
plt.figure(figsize=(12,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlación')
plt.show()

**Pregunta:** ¿Qué indica el resultado de la correlación entre las variables?

**Respuesta:**

Correlación Operating Profit vs Total Sales = 0.96
Correlación Operating Profit vs Units Sold = 0.89

1. En ambos casos la correlación es muy alta, lo que indica que existe una relación estrecha tanto entre el Operating Profit vs Total Sales como entre Operating Profit vs Units sols
2. En ambos casos la correlación es positiva, lo que indica que "a mayor número de ventas, mayor beneficio operativo"
3. La correlación es ligeramente más alta con el número total de ventas vs el número de unidades vendidas, lo que puede indicar que importa un poco menos en el beneficio operativo la cantidad de unidades vendidas vs el total de ingresos obtenidos.

## 5. Eficacia de los métodos de venta: ¿Qué método de venta es más eficaz:  la tienda física o la tienda online?

In [ ]:
# En primer lugar vamos a comprobar los posibles valores de 'sales_method_col', que contiene el canal empleado para la venta
data['Sales Method'].unique()

In [ ]:
# Ahora vamos a comparar cada uno de los métodos de venta en relación a:
# 1. Volumen de ventas (Total Sales)
# 2. Beneficio obtenido (Operation Profit)
# 3. Margen de beneficio (Operation Margin)

#Primero agrupamos el Volumen de ventas y el Beneficio obtenido en relación a cada canal
sales_method_grouped = data.groupby('Sales Method').agg({'Total Sales': 'sum', 'Operating Profit': 'sum'})

#A continuación añadimos una columna con el margen operativo calculada a partir del Volumen de ventas y el Beneficio obtenidos anteriormente
sales_method_grouped['Operating Margin'] = sales_method_grouped['Operating Profit'] / sales_method_grouped['Total Sales']

sales_method_grouped

In [ ]:
#Representamos el volumen de ventas en cada canal

from matplotlib import pyplot as plt
sales_method_grouped['Total Sales'].plot(kind='line', figsize=(8, 4), title='Total Sales')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
#Representamos el beneficio obtenido en cada canal

from matplotlib import pyplot as plt
sales_method_grouped['Operating Profit'].plot(kind='line', figsize=(8, 4), title='Operating Profit')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
#Representamos el margen operativo obtenido en cada canal

from matplotlib import pyplot as plt
sales_method_grouped['Operating Margin'].plot(kind='line', figsize=(8, 4), title='Operating Margin')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# Set the background color and figsize
plt.figure(figsize = (10, 6))

# Create the Violin plot
sns.violinplot(x = 'Sales Method', y = 'Price per Unit', data = data, palette = 'cool')

# Set the axis and title
plt.title('Distribution de los precios de los productos según canal de venta')
plt.xlabel('Canal de Venta')
plt.ylabel('Precios unitarios de los productos')

plt.show()

In [ ]:
# Set the background color and figsize
plt.figure(figsize = (10, 6))

# Create the Violin plot
sns.violinplot(x = 'Sales Method', y = 'Units Sold', data = data, palette = 'cool')

# Set the axis and title
plt.title('Distribution de las unidades de productos vendidas según canal de venta')
plt.xlabel('Canal de Venta')
plt.ylabel('Unidades de productos vendidas')

plt.show()

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Respuesta:**

- Ventas totales: - En tienda:  356.643.750 - Online: 247.672.882 - Outlet: 295.585.493 Las ventas en tienda son las más elevadas, seguidas de las ventas outlet y online.

- Beneficios de explotación: - En tienda:  127.591.300 (aproximadamente) - Online: 96.555.180 (aproximadamente) - Outlet: 107.988.300 $ (aproximadamente). De forma similar a las ventas totales, las ventas en tienda lideran los beneficios de explotación, seguidas de las de outlet y luego de las online.

- Margen de explotación (beneficio de explotación dividido por ventas totales): - En tienda: 0,357756 (o 35,78%) - Online: 0,389850 (o 38,99%) - Outlet: 0,365337 (o 36,53%)

- En el outlet se venden productos de precio unitario menor a los que se venden en tienda física y online. En tienda física y online la distribución de precios de los productos es más parecida, aunque en tienda física parece que se venden un poco más productos más caros.

Por tanto, mientras que en tienda tenemos las ventas totales y los beneficios más altos, las ventas en línea tienen el margen operativo más alto, lo que indica una mayor rentabilidad en relación con las ventas generadas.

Conclusiones: - En términos de Ventas Totales y Beneficio de Explotación: El método de venta en tienda es el más eficaz, ya que genera las mayores ventas totales y beneficios de explotación. - En términos de margen de explotación: Las ventas en línea son las más eficaces, lo que indica que, aunque las ventas totales y el beneficio de explotación son menores que en tienda, la rentabilidad en relación con las ventas generadas es mayor.

A la hora de decidir qué método de venta es más eficaz, depende de lo que priorice la empresa. Si el objetivo es maximizar los ingresos y los beneficios totales, las ventas en tienda son más eficaces. Sin embargo, si la atención se centra en la eficiencia en términos de beneficio generado por dólar de ventas, entonces las ventas en línea son más eficaces.

In [ ]:
# Cuando dibujamos mapas con la librería plotly, es necesario que los nombres de los estados sean abreviaturas de 2 letras como NY, CA, NJ, etc en lugar de Nueva York, California, Nueva Jersey.
# Para conseguirlo usamos una biblioteca llamada "us", que nos ayuda a obtener los estados de EEUU como abreviaturas
from us import states

state_column = data['State']

# Obtenemos las abreviaturas de dos letras de los nombres de los estados
state_abbreviations = []
for state in state_column:
    try:
        # Vamos recorriendo en la columna cada nombre de estado y obtenemos la abreviatura correspondiente, que almacenamos
        # en una nueva variable state_abbreviation
        state_abbreviation = states.lookup(state).abbr
    except AttributeError:
        # En caso de que no se encuentre el estado, completamos con un None
        state_abbreviation = None
    state_abbreviations.append(state_abbreviation)

# Añadimos una nueva columna 'State Abbreviation' al dataframe con las abreviaturas de dos letras de los estados
data['State Abbreviation'] = state_abbreviations

In [ ]:
# Comprobamos que el contenido de la nueva columna es correcto
data['State Abbreviation'].head()

In [ ]:
# Comprobamos los distintos valores de la nueva columna (no hay ningún None)
data['State Abbreviation'].head()
data['State Abbreviation'].unique()

#### 4.1 Demonstrating Total Sales by State, Product, and Sales Method in the US using Choropleth Map

In [ ]:
# Eliminamos de los símbolos de moneda y conversión de "Total Sales" a numérico
data['Total Sales'] = data['Total Sales'].replace('[\$,]', '', regex=True).astype(float)

# Inicializamos la figura
fig = go.Figure()

# Obtenemos y almacenamos las listas de productos y métodos de venta existentes
products = data['Product'].unique()
sales_methods = data['Sales Method'].unique()

# Añadir trazas para cada combinación de producto y método de venta
for product in products:
    for method in sales_methods:
        filtered_df = data[(data['Product'] == product) & (data['Sales Method'] == method)]
        state_sales = filtered_df.groupby('State Abbreviation')['Total Sales'].sum().reset_index()

        fig.add_trace(
            go.Choropleth(
                locations=state_sales['State Abbreviation'],
                z=state_sales['Total Sales'],
                locationmode='USA-states',
                colorscale='Viridis',
                name=f"{product} - {method}",
                showscale=True,
                visible=False  # Inicoamnete todas las trazas se ocultan
            )
        )

# Actualizamos el diseño de la figura con menús desplegables
product_buttons = [
    {'label': product, 'method': 'update', 'args': [{'visible': [trace.name.startswith(product) for trace in fig.data]}]}
    for product in products
]

sales_method_buttons = [
    {'label': method, 'method': 'update', 'args': [{'visible': [method in trace.name for trace in fig.data]}]}
    for method in sales_methods
]

fig.update_layout(
    updatemenus=[
        {'buttons': product_buttons, 'direction': 'down', 'showactive': True, 'x': 0.25, 'xanchor': 'left', 'y': 1.15, 'yanchor': 'top'},
        {'buttons': sales_method_buttons, 'direction': 'down', 'showactive': True, 'x': 0.75, 'xanchor': 'left', 'y': 1.15, 'yanchor': 'top'}
    ],
    geo=dict(scope='usa'),
    title="Total Sales by State, Product, and Sales Method"
)

# Mostrar inicialmente el primer producto y el primer método de venta
if fig.data:
    fig.data[0].visible = True

fig.show()

## 6. Optimización del precio: ¿Existe un rango de precios concreto que consiga mejores ventas que otros?

In [ ]:
# Definimos las franjas de precios que vamos a utilizar
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130]

# Creamos una nueva columna 'Price Range' que contendrá los rangos de precios a partir de las franjas anteriores
data['Price Range'] = pd.cut(data['Price per Unit'], bins)

# Agrupamos el volumen total de ventas de cada una de las franjas de precios que hemos creado anteriormente
sales_by_price_range = data.groupby('Price Range')['Total Sales'].sum().reset_index()

# Ordenamos el resultado en función del volumen de ventas de cada franja, de mayor a menor
sorted_sales_by_price_range = sales_by_price_range.sort_values(by='Total Sales', ascending=False)

# Presentamos el resultado obtenido
sorted_sales_by_price_range

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Respuesta:**
1.	Las ventas totales más elevadas se sitúan en la franja de precios de 40 a 50 dólares (220.666.307 dólares), lo que indica que los productos con precios dentro de esta franja son muy populares o tienen una gran demanda.
2.	A continuación se sitúan los precios de 50-60 dólares (210.865.002 dólares) y 60-70 dólares (190.679.285 dólares). Estos rangos también parecen ser eficaces en términos de ventas.
3. A medida que el precio aumenta por encima de los 70 dólares, se produce un descenso notable de las ventas totales. Por ejemplo, el rango de 70 a 80 dólares experimenta un descenso significativo hasta los 52.787.579 dólares.
4. Las cifras de ventas más bajas se observan en el extremo inferior (de 0 a 10 dólares) y superior (de 110 a 130 dólares) del espectro de precios.

#### Insights:
- Los productos de precio medio (concretamente, entre 40 y 70 dólares) son los que parecen conseguir mejores ventas. Esto podría indicar que existe un punto óptimo de precios en el que los clientes están más dispuestos a comprar.

- Los productos de precio muy bajo (menos de 20 dólares) y los de precio más alto (más de 90 dólares) registran ventas significativamente más bajas. Esto puede deberse a varios factores, como el valor percibido, la calidad del producto, el segmento de clientes objetivo o la disponibilidad.

- La gama de 120 a 130 dólares no registra ventas, lo que indica una falta de productos en esta gama o la ausencia de demanda de productos a este precio.

#### Conclusion:
- Una gama de precios específica, en particular entre 40 y 70 dólares, sí consigue mejores ventas en comparación con otras gamas.

- Para entender las razones de esta tendencia se necesitaría un análisis cualitativo adicional, como las preferencias de los clientes, los tipos de productos dentro de cada gama de precios, la competencia en el mercado y las estrategias promocionales.

- Esta información puede ser útil para las estrategias de fijación de precios, la gestión de inventarios y las campañas de marketing, centrándose más en las gamas de precios que han mostrado mayores volúmenes de ventas.


## 7. Optimización del portfolio de productos: ¿Qué productos son más rentables en cada ubicación?

In [ ]:
# Agrupamos los datos por localización (Ciudad) y producto, para quedarnos con el volumen total de ventas y el margen operativo total
# en cada par Ciudad/Producto
grouped_data = data.groupby(['City', 'Product']).agg({'Operating Profit': 'sum', 'Total Sales': 'sum'}).reset_index()

# Calculamos el margen de beneficio para cada producto en cada una de las ubicaciones
grouped_data['Profit Margin'] = grouped_data['Operating Profit'] / grouped_data['Total Sales']

# Ordenamos dentro de cada localización, ordenamos los productos con mayor margen de beneficio
grouped_data.sort_values(by=['City', 'Profit Margin'], ascending=[True, False], inplace=True)

# Visualizamos el Top 5 de productos con mejor margen dentro de cada ciudad
top_products_by_city = grouped_data.groupby('City').head(5)

top_products_by_city

In [ ]:
# Agrupamos los datos por localización (Ciudad) y producto, para quedarnos con el volumen total de ventas en cada par Ciudad/Producto
grouped_data = data.groupby(['Product', 'City'])['Total Sales'].sum().reset_index()

# Para cada producto, buscamos la ciudad en la que se comporta mejor/peor
best_worst_performing_cities = []

for product in grouped_data['Product'].unique():
    product_data = grouped_data[grouped_data['Product'] == product]
    best_city = product_data[product_data['Total Sales'] == product_data['Total Sales'].max()]
    worst_city = product_data[product_data['Total Sales'] == product_data['Total Sales'].min()]
    best_worst_performing_cities.extend([best_city, worst_city])

# Concatenamos los resultados obtenidos en un único y nuevo dataframe
best_worst_performing_cities_df = pd.concat(best_worst_performing_cities).reset_index(drop=True)

# Visualizamos los resultados
print("Best and Worst Performing Cities for Each Product:")
best_worst_performing_cities_df

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Respuesta:**

**Perspectivas de rentabilidad

En Albany, "Ropa de mujer" ('Women's Apparel') y "Calzado de calle para hombre" ('Men's Street Footwear') son muy rentables, con márgenes de beneficio en torno al 49% y el 46%, respectivamente.
Wilmington muestra una tendencia diferente, con "Ropa de mujer", "Calzado deportivo de mujer" y "Calzado de calle de mujer" ('Women's Apparel', 'Women's Athletic Footwear', and 'Women's Street Footwear') a la cabeza de la rentabilidad, cada una en torno al 40%.

**Perspectivas de rendimiento de las ventas

Nueva York destaca en las ventas de "Ropa de hombre", "Calzado deportivo de hombre" y "Calzado deportivo de mujer" ('Men's Apparel', 'Men's Athletic Footwear', and 'Women's Athletic Footwear'), lo que indica un mercado fuerte para estos productos.
Charleston y San Francisco obtienen los mejores resultados en "Calzado de calle para hombre" y "Calzado de calle para mujer" ('Men's Street Footwear' and 'Women's Street Footwear'), respectivamente.
Omaha registra sistemáticamente las ventas más bajas en varios productos, lo que sugiere una penetración o demanda limitadas en el mercado.

**Implicaciones generales

La popularidad y rentabilidad de los productos varía significativamente según la ciudad, lo que indica la necesidad de estrategias de marketing y ventas adaptadas a cada lugar.
El contraste de tendencias entre ciudades como Albany, Wilmington y Omaha pone de manifiesto la diversidad de preferencias de los consumidores y subraya la importancia de adoptar un enfoque local en las estrategias de marketing y ventas.

## 8. Oportunidades de expansión del mercado: Evaluar las tiendas con mejores y peores resultados en función de su ubicación.

In [ ]:
# Copiamos el dataframe en una nueva variable 'df'
df = pd.DataFrame(data)

# Agrupamos los datos por Ciudad 'City' y Nombre de la tienda o del minorista 'Retailer'
grouped_data = df.groupby(['City', 'Retailer']).agg({
    'Total Sales': 'sum',
    'Operating Profit': 'sum'
}).reset_index()

# Calculamos el margen de beneficio para cada pareja Ciudad/Retailer
grouped_data['Profit Margin'] = grouped_data['Operating Profit'] / grouped_data['Total Sales']

# Ordenamos en función del volumen total de ventas y del margen de beneficio
grouped_data = grouped_data.sort_values(by=['City', 'Total Sales', 'Profit Margin'], ascending=[True, False, False])

# Presentamos los resultados
print("Store Performance by City:")
grouped_data

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Respuesta:**

Los datos se han agregado para mostrar el rendimiento de diferentes tiendas minoristas en varias ciudades.
Las métricas clave consideradas son las ventas totales, el beneficio de explotación y el margen de beneficio (Total Sales, Operating Profit, and Profit Margin).
Las tiendas están agrupadas por ciudad y minorista, lo que ofrece una visión clara del rendimiento de cada tienda en su respectiva ubicación.

- **Mejores resultados:** En Albany, "West Gear" es la empresa con mejores resultados, con unas ventas totales de unos 20,7 millones de dólares y un margen de beneficios del 38,9%. Esta cifra es significativamente superior a la de "Kohl's" en la misma ciudad, que tiene unas ventas totales de unos 3,7 millones de dólares con un margen de beneficios del 37%.
En Albuquerque, "Kohl's" lidera con unas ventas totales de 17,1 millones de dólares y un margen de beneficio de aproximadamente el 33,9%.

- **Comparación de los resultados de las tiendas:**: En las ciudades con varios minoristas en la lista, como Albuquerque y Wichita, hay una diferencia notable tanto en las ventas como en los márgenes de beneficio entre las tiendas. Por ejemplo, en Albuquerque, "Kohl's" no solo tiene mayores ventas totales, sino también un margen de beneficios ligeramente inferior en comparación con "Sports Direct".

- **Análisis de márgenes de beneficio:**Los márgenes de beneficio varían según las ciudades y las tiendas. Mientras que "West Gear" en San Luis tiene un alto margen de beneficio del 40,1%, otras tiendas como "Amazon" en Anchorage muestran márgenes de beneficio más bajos (31%).

- **Ventas vs. Márgenes de beneficio:** No siempre existe una correlación directa entre ventas elevadas y altos márgenes de beneficio. Por ejemplo, "Kohl's" en Wichita tiene mayores ventas que "Foot Locker" en la misma ciudad, pero sus márgenes de beneficio son bastante parecidos (35,3% para Kohl's frente a 34,9% para Foot Locker).

- **Variaciones regionales:** Los datos indican posibles variaciones regionales en el rendimiento de las tiendas. Las tiendas de distintas ciudades con la misma marca (como "Kohl's" en Albany frente a Albuquerque) muestran distintos niveles de ventas y márgenes de beneficio, lo que sugiere que la ubicación es un factor importante en el rendimiento del comercio minorista.

- **Conclusion:** Este análisis proporciona información valiosa sobre el rendimiento de las tiendas minoristas en diferentes ciudades, destacando la importancia de la ubicación en las ventas y la rentabilidad. Los datos pueden utilizarse para identificar las zonas fuertes y débiles, optimizar las operaciones y elaborar estrategias de expansión o mejora del mercado.

## 9. Análisis de series temporales: Investigar si ha habido una tendencia de ventas constante a lo largo del tiempo o alguna tendencia mensual o estacional notable.

In [ ]:
# Agrupamos por mes para calcular las ventas totales cada mes
monthly_sales = data.groupby(data['Invoice Date'].dt.to_period('M'))['Total Sales'].sum()

# Convertimos el índice (que es un tipo de 'Periodo') en objeto de tipo DataTime
monthly_sales.index = monthly_sales.index.to_timestamp()

# Definimos el estilo de la figura
sns.set(style="whitegrid")

# Representamos la tendencia de ventas en el tiempo
plt.figure(figsize=(12, 7))
plt.plot(monthly_sales.index, monthly_sales.values, marker='o', color='teal', linestyle='-', linewidth=2)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales', fontsize=12)
plt.title('Monthly Sales Trends', fontsize=14)
plt.xticks(rotation=0)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.tight_layout()
plt.show()

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Respuesta:**

#### Resumen:

Los datos de ventas mensuales de enero de 2020 a enero de 2022 muestran una variabilidad significativa con picos notables alrededor de abril de 2021 y aumentos consistentes en diciembre y abril de cada año, potencialmente correlacionados con temporadas de vacaciones y promociones de ventas. Los datos muestran una tendencia general al alza en las ventas durante el periodo de dos años, a pesar de algunos descensos notables, especialmente en julio de 2020 y octubre de 2021.

#### Tips:

Los patrones estacionales observados, con picos y caídas de ventas, ponen de relieve la importancia de comprender el comportamiento de compra de los clientes y el impacto de la estacionalidad en las ventas. Aprovechando esta información, las empresas pueden adaptar sus esfuerzos de marketing, gestión de inventarios y asignación de recursos a los periodos previstos de mayor demanda, garantizando así que se aprovechen al máximo las oportunidades durante las temporadas altas. Este enfoque estratégico puede conducir a una mejor preparación para los picos de demanda, a una mayor satisfacción del cliente y, en general, a un mayor rendimiento de la empresa.







# 10. Análisis predictivo de ventas: Desarrollo de una previsión de ventas mensuales.

In [ ]:
# Creamos un modelo de tipo ARIMA a partir de los datos de ventas mensuales obtenidos en el punto anterior
model = ARIMA(monthly_sales, order=(2,1,2))  # Order: (p, d, q)
# Entrenamos el modelo
model_fit = model.fit()

# Realizamos la predicción con el modelo creado para los siguientes 12 meses
forecast_steps = 12
forecast = model_fit.forecast(steps=forecast_steps)

# Visualice los datos de ventas reales y los valores previstos para los próximos meses..
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales.index, monthly_sales.values, marker='o', label='Actual Sales')
plt.plot(pd.date_range(start=monthly_sales.index[-1], periods=forecast_steps+1, freq='M')[1:], forecast, marker='o', label='Forecasted Sales')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.title('Monthly Sales Forecast')
plt.legend()
plt.xticks(rotation=0)
plt.grid(True)
plt.show()

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Respuesta:**

# 11. ¿Cómo es la distribución de ventas según el precio unitario del producto? ¿Existen outliers?

In [ ]:
# Visualizar outliers utilizando un boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=data['Price per Unit'])
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para una mejor visualización
plt.title('Diagrama de Caja para Detectar Outliers')
plt.show()

**Pregunta:** ¿Qué podemos interpretar a partir de los resultados obtenidos?

**Pregunta:**

- Como ya hemos visto, la media del precio de los productos vendidos se encuentra ligeramente por encima de 40 dólares, con la mayor parte entre 35 y 65 dólares

- Existen outliers de productos con precios unitarios muy superiores al resto